<a href="https://colab.research.google.com/github/nouf7/ML/blob/main/ML_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1
For this homework will be dealing with (CIFAR10) dataset. It has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.


---



**First Task:**

1. Dataset Loading

  1.1 : Start by loading the dataset (see the PyTorch documentation for guidance)

  1.2 : Next, split the data into train, validation, and test splits.

  1.3 : Normalize data to have mean 0 and standard deviation 1.

In [ ]:
import torch
import torchvision # Using torchvision, it’s extremely easy to load CIFAR10.
import torchvision.transforms as transforms # transforms used for (data augmentation, normalization, resizing...etc)
import torch.nn as nn
import torch.optim as optim


In [ ]:
cifar_mean = 0.0
cifar_std = 1.0
# Define the transformations (including normalization)
transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((cifar_mean),(cifar_std))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the sizes of each split
total_size = len(trainset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Use random_split to create the splits
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(trainset, [train_size, val_size, test_size])


In [ ]:
print(total_size, train_size, val_size, test_size)

50000 35000 7500 7500


shuffle=True: When set to True, it shuffles the data at the beginning of each epoch (an epoch is one complete pass through the entire dataset). Shuffling is important during training to ensure that the model doesn't learn any order-related patterns in the data.

In [ ]:
batch_size = 64  # You can adjust this as needed

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


Below, I'm trying to check the mean and std, but it appears is not giving mean=0, std =1
# ** NEED FIXING **




In [ ]:

# Initialize variables to accumulate channel-wise means and stds
channel_means = torch.zeros(3)
channel_stds = torch.zeros(3)

# Iterate through the dataset to calculate means and stds
for images, _ in train_loader:
    # Calculate mean and std for each channel
    batch_means = torch.mean(images, dim=(0, 2, 3))
    batch_stds = torch.std(images, dim=(0, 2, 3))

    # Accumulate batch means and stds
    channel_means += batch_means
    channel_stds += batch_stds

# Calculate the overall mean and std
total_samples = len(train_loader.dataset)
overall_mean = channel_means / total_samples
overall_std = channel_stds / total_samples

print("means:", overall_mean)
print("stds:", overall_std)


means: tensor([0.0077, 0.0075, 0.0070])
stds: tensor([0.0039, 0.0038, 0.0041])




---


**Second Task:**

**2. Training Your Classifiers**
  
  2.1. MLP

  Just as demonstrated in the discussion, you can Implement a classification model for CIFAR-10 using MLP layers. You should be able to achieve 50% accuracy with three linear layers with ReLU as the activation function.

MLP (Multi-Layer Perceptron) and CNN (Convolutional Neural Network) are both types of artificial neural networks, MLPs are versatile and used for a wide range of tasks but are less effective for grid-like data like images. CNNs are specialized for image-related tasks due to their ability to capture spatial hierarchies and local patterns efficiently. The choice between MLP and CNN depends on the nature of the data and the task you're trying to solve.

ReLU is used to make neural networks more expressive, efficient, and capable of learning complex patterns in data.


In [ ]:
# Define the MLP classifier
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.relu = nn.ReLU()  # ReLU activation function
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second fully connected layer
        self.fc3 = nn.Linear(hidden_size, num_classes)  # Output layer

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.relu(self.fc1(x))  # Apply ReLU to the first hidden layer
        x = self.relu(self.fc2(x))  # Apply ReLU to the second hidden layer
        x = self.fc3(x)  # Output layer
        return x


In [ ]:

# Define hyperparameters for the MLP classifier
input_size = 3 * 32 * 32  # CIFAR-10 images are 32x32 pixels with 3 channels (RGB)
hidden_size = 256  # Number of neurons in each hidden layer (adjust as needed)
num_classes = 10  # Number of classes in CIFAR-10
learning_rate = 0.001  # Learning rate for optimizer
num_epochs = 10  # Number of training epochs

# Create the MLP classifier model
mlp_model = MLPClassifier(input_size, hidden_size, num_classes)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)  # Adam optimizer


In [ ]:
# Training loop for the MLP classifier
for epoch in range(num_epochs):
    mlp_model.train()  # Set the model to training mode
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients from the previous batch
        outputs = mlp_model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters using gradients

    # Print training loss for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 1.8413
Epoch [2/10], Loss: 1.6109
Epoch [3/10], Loss: 1.5992
Epoch [4/10], Loss: 1.7406
Epoch [5/10], Loss: 1.5669
Epoch [6/10], Loss: 1.3715
Epoch [7/10], Loss: 1.3103
Epoch [8/10], Loss: 1.2739
Epoch [9/10], Loss: 1.4488
Epoch [10/10], Loss: 1.5876


In [ ]:
# Evaluation on the validation set
mlp_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        outputs = mlp_model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class labels
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Number of correct predictions

accuracy = 100 * correct / total  # Calculate accuracy
print(f'Validation Accuracy: {accuracy:.2f}%')  # Print validation accuracy

Validation Accuracy: 46.75%




---
**2.2: CNN**

  Try to use CNN layers in your classifier. You should be able to achieve 60% accuracy with three CNN layers.


In [ ]:
# Define the CNN classifier
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # Adjusted input size to match the output of the last convolutional layer
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
# Define hyperparameters for the CNN classifier
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Create the CNN classifier model
cnn_model = CNNClassifier(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)


In [ ]:
# Training loop for the CNN classifier
for epoch in range(num_epochs):
    cnn_model.train()  # Set the model to training mode
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients from the previous batch
        outputs = cnn_model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters using gradients

    # Print training loss for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 1.3640
Epoch [2/10], Loss: 1.2443
Epoch [3/10], Loss: 1.2754
Epoch [4/10], Loss: 0.8831
Epoch [5/10], Loss: 0.7941
Epoch [6/10], Loss: 0.8885
Epoch [7/10], Loss: 0.9736
Epoch [8/10], Loss: 0.9205
Epoch [9/10], Loss: 0.7680
Epoch [10/10], Loss: 0.9016


In [ ]:
# Evaluation on the validation set
cnn_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        outputs = cnn_model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class labels
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Number of correct predictions

accuracy = 100 * correct / total  # Calculate accuracy
print(f'Validation Accuracy: {accuracy:.2f}%')  # Print validation accuracy

Validation Accuracy: 67.67%




---



ResNet, short for **"Residual Network**," is a deep neural network architecture that was introduced in the paper titled "Deep Residual Learning for Image Recognition" by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, published in 2015. ResNet is a groundbreaking architecture known for its ability to train very deep neural networks effectively, overcoming the challenges of vanishing gradients and degradation that typically occur as networks become deeper.

**2.3. ResNet**

  There is a widely used and powerful model architecture, ResNet. Fortunately, PyTorch has a built-in implementation of it. Check its document and try to use ResNet to train your classification model. You should be able to achieve 70% accuracy easily with ResNet-18.
  ResNet is based on CNN and batch normalization layers. Batch normalization is implemented in PyTorch too (BatchNorm2D). Try to read the ResNet paper and implement ResNet by yourself. You can check the correctness of your implementation by comparing the performance of your implementation with the performance of the built-in one.

In [ ]:
# Define the ResNet-18 classifier
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        # Load pre-trained ResNet-18 (omit final classification layer)
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        num_ftrs = self.resnet18.fc.in_features
        # Add a new classification layer for CIFAR-10
        self.resnet18.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet18(x)


In [ ]:
# Define hyperparameters for the ResNet-18 classifier
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Create the ResNet-18 classifier model
resnet_model = ResNetClassifier(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)


In [29]:
# Training loop for the ResNet-18 classifier
for epoch in range(num_epochs):
    resnet_model.train()  # Set the model to training mode
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients from the previous batch
        outputs = resnet_model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters using gradients

    # Print training loss for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.9175
Epoch [2/10], Loss: 0.7376
Epoch [3/10], Loss: 0.6429
Epoch [4/10], Loss: 0.5350
Epoch [5/10], Loss: 0.4381
Epoch [6/10], Loss: 0.2442
Epoch [7/10], Loss: 0.2519
Epoch [8/10], Loss: 0.1349
Epoch [9/10], Loss: 0.2436
Epoch [10/10], Loss: 0.1108


In [30]:
# Evaluation on the validation set
resnet_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        outputs = resnet_model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class labels
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Number of correct predictions

accuracy = 100 * correct / total  # Calculate accuracy
print(f'Validation Accuracy (ResNet-18): {accuracy:.2f}%')  # Print validation accuracy

Validation Accuracy (ResNet-18): 78.93%


tmw todo:

- data augmentation
- model analysis
-questions/ observation

**3. Data Augmentation**

  Data augmentation is a technique people usually use when training a model. The main idea is to perturb the input and use the perturbed data along with the original data to train the model. This is as if we have more data to train the model. The library torchvision implemented many commonly used perturbations that can be used for data augmentation. Check the document and try to use some of them when training your classifier. Using RandomHorizontalFlip and RandomCrop, you should be able to achieve 80% accuracy easily with ResNet-18.

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop images with padding
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to mean=0 and std=1
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
# Load CIFAR-10 dataset with data augmentation for training
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

# Load CIFAR-10 dataset without data augmentation for testing
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the sizes of each split
total_size = len(trainset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Use random_split to create the splits
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(trainset, [train_size, val_size, test_size])

batch_size = 64  # You can adjust this as needed

In [ ]:
# Define the ResNet-18 classifier
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        # Load pre-trained ResNet-18 (omit final classification layer)
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        num_ftrs = self.resnet18.fc.in_features
        # Add a new classification layer for CIFAR-10
        self.resnet18.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet18(x)


In [ ]:
# Define hyperparameters for the ResNet-18 classifier
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Create the ResNet-18 classifier model
resnet_model = ResNetClassifier(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]


In [ ]:
# Training loop for the ResNet-18 classifier
for epoch in range(num_epochs):
    resnet_model.train()  # Set the model to training mode
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients from the previous batch
        outputs = resnet_model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters using gradients

    # Print training loss for this epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.8028
Epoch [2/10], Loss: 1.0316
Epoch [3/10], Loss: 0.6643
Epoch [4/10], Loss: 0.5434
Epoch [5/10], Loss: 0.5404
Epoch [6/10], Loss: 0.4818
Epoch [7/10], Loss: 0.5812
Epoch [8/10], Loss: 0.5789
Epoch [9/10], Loss: 0.5349
Epoch [10/10], Loss: 0.4171


In [ ]:
# Evaluation on the validation set
resnet_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        outputs = resnet_model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class labels
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Number of correct predictions

accuracy = 100 * correct / total  # Calculate accuracy
print(f'Validation Accuracy (ResNet-18): {accuracy:.2f}%')  # Print validation accuracy

Validation Accuracy (ResNet-18): 78.83%




---

**4. Model Analysis**

  Take a look at some examples of model predictions. When the model is incorrect, is it incorrect in reasonable ways? Look into how the model misclassifies images using e.g., a confusion matrix.

**5. Questions/Observations:**
1. Do you observe that the accuracy of the training set is usually greater than the accuracy of the testing set?
2. Do you observe that the accuracy of the testing set is usually close to the accuracy of the validation set?
3. In your experiment,do you observe your model overfit?Do you observe that ResNet-18 does not overfit as easily even though ResNet-18 has many parameters?

My Answers:

1- Yes, it's a common observation that the accuracy of the training set tends to be higher than that of the testing set. This phenomenon, known as overfitting, occurs when a model becomes overly specialized in capturing the details and noise within the training data, making it struggle to generalize effectively to new, unseen data. Addressing overfitting through techniques like regularization, data augmentation, and monitoring validation metrics is crucial for achieving better generalization and improving testing set accuracy.

2- Based on the validation accuracy results for MLP, CNN, and ResNet, it does appear that the accuracy of the testing set is generally close to the accuracy of the validation set. While the specific numbers may vary (e.g., 50% for MLP, 67% for CNN, and 78% for ResNet), the testing set accuracy tends to be in the same ballpark as the validation set accuracy. This suggests that the models are reasonably good at generalizing their performance from the validation set to the testing set, which is a positive indication of model robustness and generalization.

3-ResNet-18 has a better capacity for generalization and is less prone to overfitting compared to MLP and CNN, despite its higher parameter count. This highlights the effectiveness of the architectural innovations in ResNet-18 for handling deep networks and complex datasets like CIFAR-10.